In [ ]:
### Classification
In classification the target is a category and its conditional distribution is 
$$P({\mathbf y}|x)$$
where:
- $\mathbf{y} \in \{ C_1,\dots, C_K\}$ represents the output variable (also called target)
- $x \in \mathbb{R}^n$ represents the vector of inputs (also called features)

In classification, the goal of learning is to return an estimator 
$$h(x,\alpha)$$
of the conditional probabiity $P({\mathbf y}|x)$, where $\alpha$ denotes the set of parameters of the model $h$. 

The learning of the estimator is done  on the basis of an available input/output
training set $D_N$ made of $N$ observation pairs $(\mathbf{x}_i,y_i)$ where $x_i \in \mathbb{R}^n$. 
